<a href="https://colab.research.google.com/github/mAcvz/Stage/blob/main/run_cupy_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%%bash
touch conf.xyz

In [9]:
%%bash
mkdir RUN

In [29]:
#@title Default title text
from numba.experimental import jitclass
from cupy import zeros, ones, rint, sqrt, sum, pi
from numpy import logical_and
from cupy import random
from cupy import int32,float64
'''
spec = [
    ('lb', int32),
    ('dslice', int32),
    ('l1', int32),
    ('l2', int32),
    ('l3', int32),
    ('rho', float64), 
    ('npart', int32),
    ('m', float64), 
    ('c3', float64),
    ('c9', float64),
    ('cf3', float64),
    ('cf9', float64),
    ('rwcut', float64),
    ('ewcut', float64),
    ('zwl', float64),
    ('zwr', float64),
    ('ywb', float64),
    ('ywt', float64),
    ('zbuff', int32),
    ('ybuff', int32),
    ('fwr', float64),
    ('fwl', float64),
    ('fwt', float64),
    ('fwb', float64),
    ('fwall', float64),
    ('gforce', float64),
    ('gamma', float64),
    ('mode', int32),
    ('c6', float64), 
    ('c12', float64), 
    ('cf6', float64), 
    ('cf12', float64), 
    ('ecut', float64), 
    ('rcut', float64), 
    ('Lx', float64),
    ('Ly', float64),
    ('Lz', float64),
    ('np', int32[:]),
    ('indc', int32[:]),
    ('indcp', int32[:]),
    ('indp', int32[:]),
    ('vcx1', int32[:]),
    ('vcy1', int32[:]),
    ('vcz1', int32[:]),
    ('x', float64[:]),
    ('y', float64[:]),
    ('z', float64[:]),
    ('rx', float64[:]),
    ('ry', float64[:]),
    ('rz', float64[:]),
    ('rcx', float64[:]),
    ('rcy', float64[:]),
    ('rcz', float64[:]),
    ('px', float64[:]),
    ('py', float64[:]),
    ('pz', float64[:]),
    ('fx', float64[:]),
    ('fy', float64[:]),
    ('fz', float64[:]),
    ('etxx', float64[:]),
    ('stxx', float64[:]),
    ('styy', float64[:]),
    ('stzz', float64[:]),
    ('stxy', float64[:]),
    ('stxz', float64[:]),
    ('styz', float64[:]),
    ('ekin', float64),
    ('ene', float64),
    ('etot', float64),
    ('ept', float64),
    ('ekt', float64),
    ('pres', float64),
    ('lload', float64),
    ('rload', float64),
    ('bload', float64),
    ('tload', float64),
    ('kg', int32),
    ('gcount', float64[:,:]),
    ('r2max', float64),
    ('r2cut', float64),
    ('ldel', float64),
    ('tt', float64),
    ('heatt', float64[:,:]),
    ('hstat', float64[:,:]),
    ('enkat', float64[:,:]),
    ('jmazt', float64[:,:]),
    ('gzt', float64[:,:]),
    ('jezt', float64[:,:]),
    ('enet', float64[:,:])
]
'''

#from numpy import int32,int64,float64

class LJ :

  def __init__(self, rho, gforce, mx, my, mz, zbuffer, nstep ):
    #-start-------------------------
    self.m  = 1.   # Xe 131.29 amu = 3.29 m_Ar   # Kr 83.80 amu = 2.1 m_Ar
    eps     = 1.   # Xe 204 K = 1.72 eps_Ar      # Kr 165 K = 1.39 eps_Ar
    sig     = 1.   # Xe 0.3975 nm =1.17 sigma_Ar # Kr 0.3633 nm = 1.07 sigma_Ar
    # potential cut-off: 3*sigma or an integer number of unit cell a 
    # WCA cut-off in the minimum
    # self.r2cut=2.**(1./3.)*max(sig1,sig2)**2
    # standard cut-off for LJ systems
    a = (4/rho)**(1./3.)
    self.dslice = 2  # analysis over slices of width a 
    cell_lenght = self.dslice*a
    self.rcut = 3.*sig
    if (cell_lenght < self.rcut):
        self.rcut = cell_lenght
    self.lb   =  mz*self.dslice
    self.l1   =  mx*self.dslice
    self.ybuff= 2  ### assume one lower and one higher additional box in input my
    self.l2   = (my-self.ybuff)*self.dslice 
    self.zbuff= zbuffer  ### additional space for moving wall included in input mz
    self.l3   = (mz-self.zbuff)*self.dslice
    self.npart= 4*self.l1*self.l2*self.l3
    self.Lx   = mx * cell_lenght
    self.Ly   = my * cell_lenght
    self.Lz   = mz * cell_lenght
    # density estimate 
    self.rho  = self.npart/(mx*(my-self.ybuff)*(mz-self.zbuff)*cell_lenght**3)
    # wall potential epsilon=sigma=1
    self.c3  = 4.0*self.rho*pi/6.
    self.c9  = self.c3*self.rho*pi/45
    self.cf3 = 3.*self.c3
    self.cf9 = 9.*self.c9
    ## purely repulsive: cut off at potential minimum 
    self.rwcut = (3.*self.c9/self.c3)**(1./6.)
    ## positiong wall
    self.zwr = a*(self.l3 + self.dslice - 0.25) + self.rwcut
    self.zwl = a*(self.dslice + 0.25) - self.rwcut
    self.ywt = a*(self.l2 + self.dslice - 0.25) + self.rwcut
    self.ywb = a*(self.dslice + 0.25) - self.rwcut
    self.fwr = 0.
    self.fwl = 0.
    self.fwt = 0.
    self.fwb = 0.
    self.fwall = 0.
    #self.rwcut *= 10    # uncomment for an attractive wall
    self.ewcut= - (self.c9/self.rwcut**6-self.c3)/self.rwcut**3
    self.gforce= gforce
    self.gamma = 0.5*self.Lx*self.Ly
    self.mode  = 0
    #
    ndim = self.npart
    ncells=mx*my*mz
    self.np   = zeros(    ncells, dtype=int32 )
    self.indc = zeros(self.npart, dtype=int32)
    self.indcp= zeros(      ndim, dtype=int32 )
    self.indp = zeros(  ncells+1, dtype=int32)
    # indexing neighbour cells of selected one
    self.vcx1 = zeros(27, dtype=int32)
    self.vcy1 = zeros(27, dtype=int32)
    self.vcz1 = zeros(27, dtype=int32)
    # indexing neighbour cells of selected one
    k = 0
    self.vcx1 = zeros(27, dtype=int32)
    self.vcy1 = zeros(27, dtype=int32)
    self.vcz1 = zeros(27, dtype=int32)
    for i in range(-1,2):
        for j in range(-1,2):
            for l in range(-1,2):
                self.vcx1[k]= i
                self.vcy1[k]= j
                self.vcz1[k]= l
                k+=1
    # N initial guess of average number of particles for dimensioning
    self.x       = zeros( ndim )
    self.y       = zeros( ndim )
    self.z       = zeros( ndim )
    self.rx      = zeros( ndim )
    self.ry      = zeros( ndim )
    self.rz      = zeros( ndim )
    self.rcx     = zeros( ndim )
    self.rcy     = zeros( ndim )
    self.rcz     = zeros( ndim )
    self.px      = zeros( ndim )
    self.py      = zeros( ndim )
    self.pz      = zeros( ndim )
    self.fx      = zeros( ndim )
    self.fy      = zeros( ndim )
    self.fz      = zeros( ndim )
    self.etxx    = zeros( ndim )
    self.stxx    = zeros( ndim )
    self.styy    = zeros( ndim )
    self.stzz    = zeros( ndim )
    self.stxy    = zeros( ndim )
    self.stxz    = zeros( ndim )
    self.styz    = zeros( ndim )
    # dimensioning array for nemd
    self.heatt = zeros( (      3,nstep+1), dtype=float64 )
    self.hstat = zeros( (self.lb,nstep+1), dtype=float64 )
    #self.hstbt = zeros( (self.lb,nstep+1), dtype=float64 )
    self.enkat = zeros( (self.lb,nstep+1), dtype=float64 )
    #self.enkbt = zeros( (self.lb,nstep+1), dtype=float64 )
    self.jmazt = zeros( (self.lb,nstep+1), dtype=float64 )
    self.gzt   = zeros( (self.lb,nstep+1), dtype=float64 )
    self.jezt  = zeros( (self.lb,nstep+1), dtype=float64 )
    self.enet  = zeros( (self.lb,nstep+1), dtype=float64 )
    #
    self.kg      = 512
    self.gcount  = zeros( (self.kg,3) )
    self.ekin    = 0.0
    self.ene     = 0.0
    self.etot    = 0.0
    #
    self.tt      = 0.0
    self.ekt     = 0.0
    self.ept     = 0.0
    self.pres    = 0.0
    self.lload   = 0.0
    self.rload   = 0.0
    self.tload   = 0.0
    self.bload   = 0.0
    #
    rmax = min( (self.Lx, self.Ly, self.Lz) )/2.
    rmax = self.rcut # to compute gdr in calcener
    self.ldel  = rmax/self.kg
    self.r2max = rmax * rmax
    self.r2cut = self.rcut**2
    # particles N = Na + Nb
    N  = self.npart
    # a-a interaction
    sig6     = sig**6
    self.c6  = 4.0*eps*sig6;
    self.c12 = self.c6*sig6;
    self.cf12= 12.*self.c12;
    self.cf6 = 6.*self.c6;
    self.ecut= - (self.c12/self.r2cut**3-self.c6)/self.r2cut**3;
    #
    # RANDOM WITH FIXED SEED 
    self.rng = random.default_rng(12345)
    
    
  def fcc(self):
    ax = self.Lx/self.l1
    ay = self.Ly/(self.l2 + self.ybuff*self.dslice)
    az = self.Lz/(self.l3 + self.zbuff*self.dslice) 
    print( "# lattice lenghts (ax,ay,az) =", (ax, ay, az) )
    print( "# (mx, my, mz) =", (self.l1,self.l2, self.l3) )
    mm = self.l1*self.l2*self.l3
    natom = 4*mm  
    print( "# number of lattice cells =", mm )
    print( "# number of particles =" , natom )
    print( "# md-box sides [Lx, Ly, Lz ]=", (self.Lx, self.Ly, self.Lz) )
    j  = 0
    xi = 0.25*ax
    yi = self.dslice*ay + 0.25*ay
    zi = self.dslice*az + 0.25*az
    delta=0.005
    rrx = self.rng.normal(0., delta, natom)
    rry = self.rng.normal(0., delta, natom)
    rrz = self.rng.normal(0., delta, natom)
    #with open("fcc.txt", "w") as f:
    for nx in range(self.l1) :
        for ny in range(self.l2) :
            for nz in range(self.l3) :
                self.x[j] = xi + ax*nx + rrx[j]
                self.y[j] = yi + ay*ny + rry[j]             
                self.z[j] = zi + az*nz + rrz[j]
                #f.write( "  %d   %8.3f   %8.3f   %8.3f \n" % (j, self.x[j], self.y[j], self.z[j]) )
                #print( (j, self.x[j], self.y[j], self.z[j]) )
                j +=1
                self.x[j] = xi + ax*nx + rrx[j] + 0.5*ax
                self.y[j] = yi + ay*ny + rry[j] + 0.5*ay     
                self.z[j] = zi + az*nz + rrz[j]
                #f.write( "  %d   %8.3f   %8.3f   %8.3f \n" % (j, self.x[j], self.y[j], self.z[j]) )
                #print( (j, self.x[j], self.y[j], self.z[j]) )
                j +=1
                self.x[j] = xi + ax*nx + rrx[j] + 0.5*ax
                self.y[j] = yi + ay*ny + rry[j]             
                self.z[j] = zi + az*nz + rrz[j] + 0.5*az
                #f.write( "  %d   %8.3f   %8.3f   %8.3f \n" % (j, self.x[j], self.y[j], self.z[j]) )
                #print( (j, self.x[j], self.y[j], self.z[j]) )
                j +=1
                self.x[j] = xi + ax*nx + rrx[j] 
                self.y[j] = yi + ay*ny + rry[j] + 0.5*ay            
                self.z[j] = zi + az*nz + rrz[j] + 0.5*az
                #f.write( "  %d   %8.3f   %8.3f    %8.3f \n" % (j, self.x[j], self.y[j], self.z[j]) )
                #print( (j, self.x[j], self.y[j], self.z[j]) )
                j +=1
    print( "# end of initial fcc lattice construction npart =",j)
    #

  def cells(self, mx, my, mz, N ):
    # nb: reduced coordinates for orthorombic box, option with/without PBC
    self.rx  = self.x/self.Lx
    self.rx -= rint(self.rx)  # si periodicity along x
    self.ry  = self.y/self.Ly
    #self.ry -= rint(self.ry) # no periodicity along y
    self.rz  = self.z/self.Lz
    #self.rz -= rint(self.rz) # no periodicity along z
    #
    ncells=mx*my*mz
    self.np[:] = 0  # zeros(ncells, dtype=int32)
    #self.indc = zeros(N, dtype=int32)
    for i in range(N):
        vcx=int(mx*(self.rx[i]+0.5)) #PBC no->int(my*(self.ry[i]    ))
        vcy=int(my*(self.ry[i]    )) #PBC si->int(my*(self.ry[i]+0.5))
        vcz=int(mz*(self.rz[i]    )) #PBC si->int(mz*(self.rz[i]+0.5))
        # cell index
        c = mz*(my*vcx+vcy)+vcz
        self.indc[i]=c
        self.np[c] += 1
    self.indp[0] = 0 # zeros(ncells+1, dtype=int32)
    for c in range(0,ncells) :
        self.indp[c+1] = self.indp[c] + self.np[c]
    for i in range(N):
        c=self.indc[i]
        self.rcx[self.indp[c]] = (self.rx[i]+0.5)*self.Lx #PBC no->(self.rx[i]    )*self.Lx 
        self.rcy[self.indp[c]] = (self.ry[i]    )*self.Ly #PBC si->(self.ry[i]+0.5)*self.Ly 
        self.rcz[self.indp[c]] = (self.rz[i]    )*self.Lz #PBC si->(self.rz[i]+0.5)*self.Lz 
        self.indcp[self.indp[c]] = i
        self.indp[c] += 1
    # need to reconstruct index list
    self.indp[0]=0
    for c in range(0,ncells) :
        self.indp[c+1] = self.indp[c] + self.np[c]

  def calcener(self, mx, my, mz, N ) :
    # zeroing
    # nb: rectangular PBC
    ene=0.
    vip=0.
    e1xx = zeros(N)
    s1xx = zeros(N)
    s1yy = zeros(N)
    s1zz = zeros(N)
    s1xy = zeros(N)
    s1xz = zeros(N)
    s1yz = zeros(N)
    f1x= zeros(N)
    f1y= zeros(N)
    f1z= zeros(N)
    # Loop over Cells
    for vcx in range(mx):
        for vcy in range(my):
            for vcz in range(mz):
                c = mz*(my*vcx+vcy)+vcz  
                # loop over particles inside selected cell
                '''
                    none
                '''
                # loop over particles in neighbour cells 
                for k in range(27) :
                    wcx=vcx + self.vcx1[k]
                    wcy=vcy + self.vcy1[k]
                    wcz=vcz + self.vcz1[k]
                    # Periodic boundary conditions 
                    shiftx = 0.
                    if (wcx == -1) :
                        shiftx =-self.Lx
                        wcx = mx-1
                    elif (wcx==mx) :
                        shiftx = self.Lx
                        wcx = 0
                    shifty = 0.
                    #if (wcy == -1) :
                    #    shifty =-self.Ly
                    #    wcy = my-1
                    #elif (wcy==my) :
                    #    shifty = self.Ly
                    #    wcy = 0
                    shiftz = 0.
                    #if (wcz == -1) :
                    #    shiftz =-self.Lz
                    #    wcz = mz-1
                    #elif (wcz==mz) :
                    #    shiftz = self.Lz
                    #    wcz = 0
                    c1 = mz*(my*wcx+wcy)+wcz

                    #
                    for i in range(self.indp[c],self.indp[c+1]):
                        #
                        dx = zeros(self.indp[c1+1]-self.indp[c1])
                        dy = zeros(self.indp[c1+1]-self.indp[c1])
                        dz = zeros(self.indp[c1+1]-self.indp[c1])
                        r2 = zeros(self.indp[c1+1]-self.indp[c1])

                        # vir = 0. # to vectorize closing if after marks
                        dx[:] = self.rcx[i] - (self.rcx[self.indp[c1]:self.indp[c1+1]] + shiftx)
                        dy[:] = self.rcy[i] - (self.rcy[self.indp[c1]:self.indp[c1+1]] + shifty)
                        dz[:] = self.rcz[i] - (self.rcz[self.indp[c1]:self.indp[c1+1]] + shiftz)
                        r2[:] = dx[:]*dx[:] + dy[:]*dy[:] + dz[:]*dz[:] 
                        #
                        # creating a vector mask 
                        mk =  logical_and(r2 < self.r2cut, r2 > 1e-10 )
                        tr_dim = sum(mk)
                        #
                        rr2 = zeros(tr_dim)
                        rr6 = zeros(tr_dim)
                        ej  = zeros(tr_dim)
                        vir = zeros(tr_dim)
                        rr2[:] = 1./r2[mk]
                        rr6[:] = rr2[:]*rr2[:]*rr2[:]
                        #
                        ej[:]  = (self.c12*rr6 -self.c6)*rr6 + self.ecut
                        ene += sum(ej[:])
                        vir[:] = (self.cf12*rr6-self.cf6)*rr6
                        vip += sum(vir[:])
                        # observable --> energy current
                        e1xx[i] = sum(ej[:])
                        
                        # forces
                        vir[:] *= rr2[:]
                        f1x[i] += sum(vir*dx[mk])
                        
                        f1y[i] += sum(vir*dy[mk])
                        
                        f1z[i] += sum(vir*dz[mk])
                        
                        # observable --> stress tensor
                        s1xx[i] += sum(vir*dx[mk]*dx[mk])
                        
                        s1yy[i] += sum(vir*dy[mk]*dy[mk])
                        
                        s1zz[i] += sum(vir*dz[mk]*dz[mk])
                        
                        s1xy[i] += sum(vir*dx[mk]*dy[mk])
                        
                        s1xz[i] += sum(vir*dx[mk]*dz[mk])
                        
                        s1yz[i] += sum(vir*dy[mk]*dz[mk])
                        
            
                
    # final reordering of atomic forces , energies and stresses
    for i in range(N): 
        self.fx[self.indcp[i]]   = f1x[i]
        self.fy[self.indcp[i]]   = f1y[i]
        self.fz[self.indcp[i]]   = f1z[i]
        self.etxx[self.indcp[i]] = e1xx[i]
        self.stxx[self.indcp[i]] = s1xx[i]
        self.styy[self.indcp[i]] = s1yy[i]
        self.stzz[self.indcp[i]] = s1zz[i]
        self.stxy[self.indcp[i]] = s1xy[i]
        self.stxz[self.indcp[i]] = s1xz[i]
        self.styz[self.indcp[i]] = s1yz[i]   
        
    return ( 0.5*ene, 0.5*vip )
    
  def calcexyz(self, N) :
    # zeroing
    ewg= 0.
    fwl= 0.
    fwr= 0.
    fwt= 0.
    fwb= 0.
    # loop over particles inside selected cell
    for i in range(N):
        self.fy[i] += self.gforce
        ewg -= self.gforce*self.y[i]
        # top & bottom walls along y
        dyt = self.ywt  - self.y[i] 
        dyb = self.y[i] - self.ywb
        if(dyt < self.rwcut) :
            rr1 = 1./dyt
            rr3 = rr1*rr1*rr1
            rr6 = rr3*rr3
            ej  = (self.c9*rr6 -self.c3)*rr3 + self.ewcut
            fyt = (self.cf9*rr6 -self.cf3)*rr3*rr1
            ewg+= ej
            fwt+= fyt
            # force on particle
            self.fy[i] -= fyt
        elif(dyb < self.rwcut) :
            rr1 = 1./dyb
            rr3 = rr1*rr1*rr1
            rr6 = rr3*rr3
            ej  = (self.c9*rr6 -self.c3)*rr3 + self.ewcut
            fyb = (self.cf9*rr6 -self.cf3)*rr3*rr1
            ewg+= ej
            fwb-= fyb
            # force on particle
            self.fy[i] += fyb
        # left & right walls along z
        dzr = self.zwr  - self.z[i] 
        dzl = self.z[i] - self.zwl
        if(dzr < self.rwcut) :
            rr1 = 1./dzr
            rr3 = rr1*rr1*rr1
            rr6 = rr3*rr3
            ej  = (self.c9*rr6 -self.c3)*rr3 + self.ewcut
            fzr = (self.cf9*rr6 -self.cf3)*rr3*rr1
            ewg+= ej
            fwr+= fzr
            # forces
            self.fz[i] -= fzr
        elif(dzl < self.rwcut) :
            rr1 = 1./dzl
            rr3 = rr1*rr1*rr1
            rr6 = rr3*rr3
            ej  = (self.c9*rr6 -self.c3)*rr3 + self.ewcut
            fzl = (self.cf9*rr6 -self.cf3)*rr3*rr1
            ewg+= ej
            fwl-= fzl
            # forces
            self.fz[i] += fzl
        # contributions to atomic stress tensor (to be added)
    return( ewg, fwr, fwl, fwt, fwb)


  def eqmdi(self, N, mx, my, mz, kt, pas, mode):
    # initial evaluation of forces, energies and virials
    self.cells(mx, my, mz, N)
    (enep, virial) = self.calcener(mx, my, mz, N)
    print("# Test intra (pot.energy, virial) \n", (enep,virial) )
    (ewg, self.fwr, self.fwl, self.fwt, self.fwb)  = self.calcexyz(N)
    print("# Test pot.energy: W+G, forces: (r, l, t, b) \n", ewg, (self.fwr, self.fwl, self.fwt, self.fwb) )
    if mode == 2 : 
        # andersen thermostats: velocity sampling from maxwellian
        pstd = sqrt(self.m*kt)
        self.px[0:N] = self.rng.normal(0., pstd, N)
        self.py[0:N] = self.rng.normal(0., pstd, N)
        self.pz[0:N] = self.rng.normal(0., pstd, N)
        vcmx = sum(self.px)
        vcmy = sum(self.py)
        vcmz = sum(self.pz)
        self.px[0:N]-= vcmx/N
        self.py[0:N]-= vcmy/N
        self.pz[0:N]-= vcmz/N
        print("# velocities sampled from maxwell distribution at timestep" , pas)
    vcmx = sum(self.px)
    vcmy = sum(self.py)
    vcmz = sum(self.pz)
    enek = 0.5*sum(self.px**2+self.py**2+self.pz**2)/self.m 
    self.ept = 0.
    self.ekt = 0.
    self.pres= 0.
    self.rload= 0.
    self.lload= 0.
    self.tload= 0.
    self.bload= 0.
    return (enep, enek, ewg, vcmx, vcmy, vcmz)   


  def eqmdr(self, N, mx, my, mz, kt, pas, dt, freq):
    dth=0.5*dt
    for ip in range(freq):
        pas+= 1
        t   = pas*dt
        # momenta first 
        self.px[0:N] += self.fx[0:N]*dth
        self.py[0:N] += self.fy[0:N]*dth
        self.pz[0:N] += self.fz[0:N]*dth        
        # positions second
        self.x[0:N]  += dt*self.px[0:N]/self.m
        self.y[0:N]  += dt*self.py[0:N]/self.m
        self.z[0:N]  += dt*self.pz[0:N]/self.m
        if self.mode:
            self.zwr += dt*(self.fwr-self.fwall)/self.gamma
        # compute forces
        self.cells(mx, my, mz, N)
        (enep, virial) = self.calcener( mx, my, mz, N)
        (ewg, self.fwr, self.fwl, self.fwt, self.fwb) = self.calcexyz(N)
        # momenta third
        self.px[0:N] += self.fx[0:N]*dth
        self.py[0:N] += self.fy[0:N]*dth
        self.pz[0:N] += self.fz[0:N]*dth   
        # one step advanced 
        vcmx = sum(self.px)
        vcmy = sum(self.py)
        vcmz = sum(self.pz)
        enek = 0.5*sum(self.px**2+self.py**2+self.pz**2)/self.m
        self.ekt += enek
        self.ept += enep
        self.pres+= virial       
        self.rload+= self.fwr
        self.lload+= self.fwl
        self.tload+= self.fwt
        self.bload+= self.fwb
    return (t, enep, enek, ewg, vcmx, vcmy, vcmz)     


  def nemdi(self, N, mx, my, mz, kt, dkt, pas, lther, rth2, option):
    tlef = 3.*kt
    trig = 3.*(kt + dkt)
    # initial evaluation of forces, energies and virials
    self.cells(mx, my, mz, N)
    (enep, virial) = self.calcener(mx, my, mz, N)
    print("# Test intra (pot.energy, virial) \n", (enep,virial) )
    (ewg, self.fwr, self.fwl, self.fwt, self.fwb)  = self.calcexyz(N)
    print("# Test pot.energy: W+G, forces: (r, l, t, b) \n", ewg, (self.fwr, self.fwl, self.fwt, self.fwb) )
    # initializing soret thermostats: velocity rescaling
    hista,eneka,heat,gz,jmaz,jez,ene = self.therm(tlef,trig,N,lther,rth2,option)
   #
    self.heatt[:,pas] =  (heat[:])
   #
    self.jmazt[:,pas] =  ( jmaz[:])
    self.gzt[:,pas]   =  (   gz[:])
    self.jezt[:,pas]  =  (  jez[:])
   #
    self.hstat[:,pas] =  (hista[:])
    self.enkat[:,pas] =  (eneka[:])
    self.enet[:,pas]  =  (  ene[:])
#
    vcmx = sum(self.px)
    vcmy = sum(self.py)
    vcmz = sum(self.pz)
    enek = 0.5*sum(self.px**2+self.py**2+self.pz**2)/self.m
    # initializing counters and constants
    self.ept  = 0.
    self.ekt  = 0.
    self.pres = 0.
    self.rload= 0.
    self.lload= 0.
    self.tload= 0.
    self.bload= 0.
    ## print("# starting dnemd trajectory")
    ## print( "( 'pas', 'enep', 'enek', 'enet', 'heatin', 'heatout', 'vcm' )")
    ## print( (0., enep/N, enek/N, (enep+enek)/N, heat[0], heat[1], vcmx, vcmy, vcmz) )
    return (enep, enek, ewg, vcmx, vcmy, vcmz)
    
  def nemdr(self, N, mx, my, mz, kt, dkt, pas, dt, freq, lther, rth2, option):
    tlef = 3.*kt
    trig = 3.*(kt + dkt)
    dth=0.5*dt
    for ip in range(freq):
        pas+= 1
        t   = pas*dt
        # advance one step
        # momenta first 
        self.px[0:N] += self.fx[0:N]*dth
        self.py[0:N] += self.fy[0:N]*dth
        self.pz[0:N] += self.fz[0:N]*dth        
        # positions second
        self.x[0:N]  += dt*self.px[0:N]/self.m
        self.y[0:N]  += dt*self.py[0:N]/self.m
        self.z[0:N]  += dt*self.pz[0:N]/self.m
        if self.mode:
            self.zwr   += dt*(self.fwr-self.fwall)/self.gamma
        # compute forces
        self.cells(mx, my, mz, N)
        (enep, virial) = self.calcener(mx, my, mz, N)
        (ewg, self.fwr, self.fwl, self.fwt, self.fwb) = self.calcexyz(N)
        # momenta third
        self.px[0:N] += self.fx[0:N]*dth
        self.py[0:N] += self.fy[0:N]*dth
        self.pz[0:N] += self.fz[0:N]*dth   
        # thermostats: velocity rescaling
        hista,eneka,heat,gz,jmaz,jez,ene = self.therm(tlef,trig,N,lther,rth2,option)
        #
        self.heatt[:,pas] =  (heat[:])
        #
        self.jmazt[:,pas] =  ( jmaz[:])
        self.gzt[:,pas]   =  (   gz[:])
        self.jezt[:,pas]  =  (  jez[:])
        #
        self.hstat[:,pas] =  (hista[:])
        self.enkat[:,pas] =  (eneka[:])
        self.enet[:,pas]  =  (  ene[:])
        #
        #end inner --> single step wrap-up
        vcmx = sum(self.px)
        vcmy = sum(self.py)
        vcmz = sum(self.pz)
        enek = 0.5*sum(self.px**2+self.py**2+self.pz**2)/self.m
        self.ekt += enek
        self.ept += enep
        self.pres+= virial   
        self.rload+= self.fwr
        self.lload+= self.fwl
        self.tload+= self.fwt
        self.bload+= self.fwb
        ## if (pas)%freq==0 : 
            ## fout.write (" %8.3f %9.4f %9.4f %10.7f %8.3f %8.3f  %7.2g %7.2g %7.2g \n" % \
    ## print( (t, enep/N, enek/N, (enep+enek)/N, heat[0], heat[1], vcmx, vcmy, vcmz) )
    return (t, enep, enek, ewg, vcmx, vcmy, vcmz)
   # end of ne-md run
    
  def therm(self,tlef,trig,N,lther,rth2,option) :
    # 1 or 2 thermostats: velocity rescaling at fixed Temperature
    # plus option with constant heating flux for 10^5 step on the right side
    flux = 1.e-5*(trig-tlef)*N   ### factor 3 already in tlef,trig
    nindl= zeros(N, dtype=int32 )
    nla  = zeros(self.lb)
    eca  = zeros(self.lb)
    vx   = zeros(self.lb)
    vy   = zeros(self.lb)
    vz   = zeros(self.lb)
    jmaz = zeros(self.lb)
    jez  = zeros(self.lb)
    e    = zeros(self.lb)
    ppx  = zeros(N)
    ppy  = zeros(N)
    ppz  = zeros(N)
    mvx  = zeros(N)
    mvy  = zeros(N)
    mvz  = zeros(N)
    heat = zeros(3)
    for i in range(N) :
        li = int((self.rz[i]    )*self.lb) 
        nla[li] += 1.
        nindl[i] = li
        vx[li]+= self.px[i]
        vy[li]+= self.py[i]
        vz[li]+= self.pz[i]
    ml   = zeros(self.lb)
    for li in range(self.lb) :
        ml[li] = (nla[li]*self.m)
    for i in range(N) :
        li = nindl[i] 
        #if(ml[li]>0.):
        mvx[i]  = self.m*vx[li]/ml[li]
        ppx[i]  = self.px[i]-mvx[i]
        mvy[i]  = self.m*vy[li]/ml[li]
        ppy[i]  = self.py[i]-mvy[i]
        mvz[i]  = self.m*vz[li]/ml[li]
        ppz[i]  = self.pz[i]-mvz[i]
        eca[li]+=(ppx[i]**2 + ppy[i]**2 + ppz[i]**2) 
    eca /=self.m
    # thermostatting lther layers on the L and R side of the box
    shift=2
    lfact= ones(self.lb)
    if option==0:
        lfact[shift:shift+lther]= tlef*(nla[shift:shift+lther]-1.)/eca[shift:shift+lther]
        heat[2]                 = sum(0.5*(lfact[shift:shift+lther]-1.)*eca[shift:shift+lther])
        eca[shift:shift+lther] *= lfact[shift:shift+lther]
        lfact[shift:shift+lther]= sqrt(lfact[shift:shift+lther])
    #
    rfact= ones(self.lb)
    lth2=rth2-lther
    rfact[lth2:rth2]= trig*(nla[lth2:rth2]-1.)/eca[lth2:rth2]    
    if option==2:
        rfact[lth2:rth2]= (eca[lth2:rth2]+flux)/eca[lth2:rth2]
    heat[0] = self.zwr
    heat[1] = sum(0.5*(rfact[lth2:rth2]-1.)*eca[lth2:rth2])
    eca[lth2:rth2] *= rfact[lth2:rth2]
    rfact[lth2:rth2]= sqrt(rfact[lth2:rth2])
    # rescale velocities of particles 
    for i in range(N) :
        li = nindl[i]
        if (li > shift) and (li < shift+lther):
            ppx[i]*= lfact[li]
            ppy[i]*= lfact[li]
            ppz[i]*= lfact[li]
            self.px[i] = ppx[i]+mvx[i]
            self.py[i] = ppy[i]+mvy[i]
            self.pz[i] = ppz[i]+mvz[i]
        elif (li >= rth2-lther) and (li < rth2) :
            ppx[i]*= rfact[li]
            ppy[i]*= rfact[li]
            ppz[i]*= rfact[li]
            self.px[i] = ppx[i]+mvx[i]
            self.py[i] = ppy[i]+mvy[i]
            self.pz[i] = ppz[i]+mvz[i]
# put back all analysis
    for i in range(N) :
        self.etxx[i] += (ppx[i]**2 + ppy[i]**2 + ppz[i]**2)/self.m
    for i in range(N) :
        li = nindl[i] 
        e[li]   += self.etxx[i]
        jmaz[li]+= ppz[i]
        jez[li] += 0.5*(ppz[i]*self.etxx[i] + self.stxz[i]*ppx[i] + self.styz[i]*ppy[i] + self.stzz[i]*ppz[i])/self.m
    #    
    return(nla, eca, heat, vz, jmaz, jez, e)



In [30]:
from numpy import zeros, pi, sqrt, rint, int64, float64
from numpy import savetxt, column_stack, empty, str_
from numpy import pi
from numpy import random
from pickle import dump, load, HIGHEST_PROTOCOL

def read_input(self, N, conf_in='conf_in.b'): #, mom_in='mom_in'):
    with open(conf_in, 'rb') as ftrj:
        (Nr, pas, self.zwr, self.fwall) = load(ftrj)
        if N!=Nr :
            #print(' reading %d particle configuration from step %d' % (Nr,pas) )
        #else :
            print(' ??? reading %d particle configuration expected %d' % (Nr,N) )
        ( self.x,  self.y,  self.z ) = load( ftrj)
        ( self.px, self.py, self.pz) = load( ftrj)
    return pas

def write_input(self, N, pas, conf_out='conf_in.b'): #, mom_out='mom_in'):
    with open(conf_out, 'wb') as ftrj:
        dump( (N, pas, self.zwr, self.fwall) , ftrj, HIGHEST_PROTOCOL)
        dump( ( self.x,  self.y,  self.z ), ftrj, HIGHEST_PROTOCOL)
        dump( ( self.px, self.py, self.pz), ftrj, HIGHEST_PROTOCOL)
               
def dumpxyz(self, N, pas, dumpf):
    dx = self.x/self.Lx
    dx-= rint(dx)
    dy = self.y/self.Ly
    #dy-= rint(dy)
    dz = self.z/self.Lz
    #dz-= rint(dz)
    ar = empty(N,(str_,2))
    sig=3.4 # in Angstroem for argon   
    ar[0:N] = "Ar"
    dx *= sig*self.Lx
    dy *= sig*self.Ly
    dz *= sig*self.Lz
    dumpf.write( " %d \n" % N ) 
    dumpf.write( " %d %10.5f  %10.5f  %10.5f  %10.5f \n" % (pas, sig*self.zwl, sig*self.zwr, sig*self.ywb, sig*self.ywt) ) 
    for i in range(N):
        dumpf.write( "%s  %10.5f  %10.5f  %10.5f  %10.5f  %10.5f  %10.5f\n" % (ar[i],dz[i],dx[i],dy[i],self.pz[i],self.px[i],self.py[i]) )
            
def writexyz(self, N, filexyz='conf.xyz'):
    dx = self.x/self.Lx
    dx-= rint(dx)
    dy = self.y/self.Ly
    #dy-= rint(dy)
    dz = self.z/self.Lz
    #dz-= rint(dz)
    ar = empty(N,(str_,2))
    sig=3.4 # in Angstroem for argon   
    ar[0:N] = "Ar"
    dx *= sig*self.Lx
    dy *= sig*self.Ly
    dz *= sig*self.Lz
    rout = column_stack( (ar, dx, dy, dz) )
    savetxt(filexyz, rout, fmt=(' %s', ' %s',' %s',' %s'), \
    header=(" %d \n walls: %10.5f  %10.5f  %10.5f  %10.5f " % (N, sig*self.zwl, sig*self.zwr, sig*self.ywb, sig*self.ywt)) )          

def write_out(self, N, tstep, gdr_out='gdr.out'):
    V = zeros(self.kg) 
    r = zeros(self.kg)
    g = zeros( (self.kg) ) 
    for lm in range(self.kg) :
        V[lm] = 4./3.*pi*(self.ldel**3)*(3*lm*lm +3*lm + 1); 
        r[lm] = (lm+0.5)*self.ldel
    g[:] = self.gcount[:]/(V*(N-1)*tstep*self.rho);
    gout = column_stack( (r, g) )
    savetxt(gdr_out, gout , fmt=('%10.5g ','%12.7g'), header="    'r'         'gaa'    " )          



In [31]:
#from m_lj import LJ
#from m_write import *
from time import process_time
from  pickle import dump, load, HIGHEST_PROTOCOL
def nerun(nstep=1000,rho=0.984,mx=6,my=10,mz=20,zbuf=4,kt=0.90,dkt=0.10,fwall=2000.,gforce=0.,freq=20,lther=3,mode=3,option=1,path="./RUN/",labelf='A'):
    #
    dt=0.005
    my   += 2    # fixed buffer for vertical direction top+bottom additional boundary boxes
    mz   += zbuf # variable buffer for horizontal direction 1+zbuf additional boundary boxes 
    rth2 = 2*(mz-zbuf+1) #  lb=2*mz - shift
    pfile = path+"nemd"+labelf+".pickle"
    tfile = path+"trajectory"+labelf+".xyz"
    #
    print( "# number of boxes mx = %d, my = %d, mz = %d" % (mx, my, mz) )
    print( "# base (kinetic) temperature kt = %8.4f  delta kt= %8.4f" % (kt, dkt) )
    print( "# integration time step dt = %8.4f" % dt )
    print( "# number of time steps for this run nst = %d " % (nstep) )    
    with open(tfile, 'w') as dumpf:
        t0 = process_time()
        NE = LJ(rho, gforce, mx, my, mz, zbuf, nstep*freq)
        print("# creating NE object, cpu = %8.4f" % (process_time()-t0) )
        N = NE.npart
        rhofact = NE.lb / (NE.Lx * NE.Ly*(my-2)/my * NE.Lz)
        print( "# sides of the rectangular MD-box L = [ %8.4f %8.4f %8.4f ]" % (NE.Lx, NE.Ly, NE.Lz) )
        print( "# number of particles  N = %d" % N)
        print( "# density rho = %8.4f   (%8.4f)" % (NE.rho, rho) )
        print( "# potential cut-off radius  rcut = %8.4f *sigma" % NE.rcut)
        print( "# lateral size of slicing Deltaz = %8.4f" % (NE.Lz/NE.lb) )
        # initial equilibrated condition
        t0 = process_time()
        estep = read_input(NE, N, conf_in=path+"conf_ne.b")
        if mode > 2 :
            NE.fwall = fwall
            print(' Dynamics with fixed external wall force %f' % fwall )
            NE.mode = 1
            mode = 1
        print("# initial conditions, cpu = %8.4f" % (process_time()-t0) )
        # Non-Equilibrium run with temperature gradient
        t0 = process_time()
        pas= 0
        dumpxyz(NE, N, pas, dumpf)
        (enep, enek, ewg, vcmx, vcmy, vcmz) = NE.nemdi( N, mx, my, mz, kt, dkt, pas, lther , rth2, option)
        print("# starting dnemd trajectory\n")
        print( "     'pas'    'enep'    'enek'       'enet'  'heatl'  'heatr'    'fwl'    'fwr'    'zwr'       'vcm' ")
        etot = enek+enep+ewg
        heatr= NE.heatt[1,0]
        heatl= NE.heatt[2,0]
        print(" %9.2f %9.4f %9.4f %12.3f %8.3f %8.3f %8.2f %8.2f %8.4f %7.1g %7.2f %7.2f" % (0.,enep/N, enek/N, etot, heatl, heatr, NE.fwl, NE.fwr, NE.zwr, vcmx, vcmy, vcmz) )
        for it in range(nstep):
            (t, enep, enek, ewg, vcmx, vcmy, vcmz) = NE.nemdr( N, mx, my, mz, kt, dkt, pas, dt, freq, lther , rth2, option)
            pas += freq
            etot = enek+enep+ewg
            heatr= NE.heatt[1,pas]
            heatl= NE.heatt[2,pas]
            print(" %9.2f %9.4f %9.4f %12.3f %8.3f %8.3f %8.2f %8.2f %8.4f %7.1g %7.2f %7.2f" % (t,enep/N, enek/N, etot, heatl, heatr, NE.fwl, NE.fwr, NE.zwr, vcmx, vcmy, vcmz) )
            # writing 
            dumpxyz(NE, N, pas, dumpf)
    print( "# Non-Equilibrium run, cpu = %8.4f" % (process_time()-t0) )
    g=3*N-1  # z-component of total momentum not conserved
    nstep *= freq
    print( "# ending ne-md trajectory  <ep>=%6.1f  <ek>=%7.4f   T=%8.3f  P=%8.3f\n" % ( NE.ept/nstep, NE.ekt/nstep, 2.*NE.ekt/(nstep*g), NE.pres/nstep) )
    print( "# ending ne-md trajectory  <L-load>=%10.3f  <R-load>=%10.3f \n" % ( NE.lload/nstep, NE.rload/nstep) )
    print( "# ending ne-md trajectory  <B-load>=%10.3f  <T-load>=%10.3f \n" % ( NE.bload/nstep, NE.tload/nstep) )
    # saving with pickle
    # final positions and momenta to file conf_in.b
    write_input(NE, N, nstep, conf_out=path+"conf_ne.b")
    with open(pfile, 'wb') as ftrj:
        rhofact = NE.lb / (NE.Lx * NE.Ly*(my-NE.ybuff)/my * NE.Lz)
        dump( (dkt,rhofact,nstep) , ftrj, HIGHEST_PROTOCOL)
        dump( (NE.heatt,NE.hstat,NE.enet,NE.enkat,NE.jmazt,NE.gzt,NE.jezt), ftrj, HIGHEST_PROTOCOL)
    # backup of final conf
    write_input(NE, N, nstep, conf_out=path+"conf_ne"+labelf+".b")
    # for visualization
    writexyz(NE, N, filexyz=path+"conf_ne"+labelf+".xyz")
    return NE




In [32]:
#from m_lj import LJ
#from m_write import *
from time import process_time
from  pickle import dump, load, HIGHEST_PROTOCOL
def eqrun(nstep=1000,rho=0.984,mx=6,my=10,mz=20,zbuf=2,kt=0.90,dkt=0.0,fwall=900.,gforce=0.,freq=20,lther=3,mode=3,path="./RUN/"):
    # default option two thermostats at fixed temperatures 
    option = 0
    dt   = 0.005
    my  += 2  # fixed buffer for vertical direction top+bottom additional boundary boxes
    mz  += zbuf #  25 Xe 16
    rth2 = 2*(mz-zbuf+1) #  lb=2*mz + shift
    pfile= path+"eqmd.pickle"
    #
    print( "# total number of boxes mx = %d, my = %d, mz = %d" % (mx, my, mz) )
    print( "# base (kinetic) temperature kt = %8.4f  delta kt= %8.4f" % (kt, dkt) )
    print( "# integration time step dt = %8.4f" % dt )
    print( "# number of time steps for this run nst = %d " % (nstep) )    
    t0 = process_time()
    EQ = LJ(rho, gforce, mx, my, mz, zbuf, nstep*freq)
    print("# creating EQ object, cpu = %8.4f" % (process_time()-t0) )
    EQ.fwall = fwall
    if mode > 2 :
        EQ.mode = 1
        mode = 1
    N = EQ.npart
    rhofact = EQ.lb / (EQ.Lx * EQ.Ly*(my-2)/my * EQ.Lz)
    print( "# rectangular MD-box: [Lx,Ly,Lz] = [ %8.4f %8.4f %8.4f ]" % (EQ.Lx, EQ.Ly, EQ.Lz) )
    print( "# number of particles: N = %d" % N)
    print( "# density: rho = %8.4f   (%8.4f)" % (EQ.rho, rho) )
    print( "# potential cut-off distance: rcut = %8.4f *sigma" % EQ.rcut)
    print( "# lateral size of slicing: Deltaz = %8.4f" % (EQ.Lz /EQ.lb))
    # initial equilibrated condition
    t0 = process_time()
    estep = read_input(EQ, N, conf_in=path+"conf_eq.b")
    print("# initial conditions, cpu = %8.4f" % (process_time()-t0) )
    # Equilibrium run with zero-temperature gradient for dkt=0.
    t0 = process_time()
    pas= 0
    (enep, enek, ewg, vcmx, vcmy, vcmz) = EQ.nemdi( N, mx, my, mz, kt, dkt, pas, lther, rth2, option)
    print("# starting md trajectory\n")
    print( "     'pas'    'enep'    'enek'       'enet'  'heatl'  'heatr'    'fwl'    'fwr'    'zwr'       'vcm' ")
    heatr= EQ.heatt[1,pas]
    heatl= EQ.heatt[2,pas]
    etot = enep+enek+ewg
    print(" %9.2f %9.4f %9.4f %12.3f %8.3f %8.3f %8.2f %8.2f %8.4f %7.1g %7.2f %7.2f" % (0.,enep/N, enek/N, etot, heatl, heatr, EQ.fwl, EQ.fwr, EQ.zwr, vcmx, vcmy, vcmz) )
    for it in range(nstep):
        (t, enep, enek, ewg, vcmx, vcmy, vcmz) = EQ.nemdr( N, mx, my, mz, kt, dkt, pas, dt, freq, lther, rth2, option)
        pas += freq
        heatr= EQ.heatt[1,pas]
        heatl= EQ.heatt[2,pas]
        etot = enep+enek+ewg
        print(" %9.2f %9.4f %9.4f %12.3f %8.3f %8.3f %8.2f %8.2f %8.4f %7.1g %7.2f %7.2f" % (t,enep/N, enek/N, etot, heatl, heatr, EQ.fwl, EQ.fwr, EQ.zwr, vcmx, vcmy, vcmz) )
    print( "# (Thermostatted run, cpu = %8.4f" % (process_time()-t0) )
    g=3*N-1  # z-component of total momentum not conserved
    nstep *= freq
    print( "# ending md trajectory  <ep>=%6.1f  <ek>=%7.4f   T=%8.3f  P=%8.3f\n" % ( EQ.ept/nstep, EQ.ekt/nstep, 2.*EQ.ekt/(nstep*g), EQ.pres/nstep) )
    print( "# ending md trajectory  <L-load>=%10.3f  <R-load>=%10.3f \n" % ( EQ.lload/nstep, EQ.rload/nstep) )
    print( "# ending md trajectory  <B-load>=%10.3f  <T-load>=%10.3f \n" % ( EQ.bload/nstep, EQ.tload/nstep) )
    # saving with pickle
    # final positions and momenta to file conf_in.b
    write_input(EQ, N, nstep, conf_out=path+"conf_ne.b")
    with open(pfile, 'wb') as ftrj:
        rhofact = EQ.lb / (EQ.Lx * EQ.Ly*(my-EQ.ybuff)/my * EQ.Lz)
        dump( (dkt,rhofact,nstep) , ftrj, HIGHEST_PROTOCOL)
        dump( (EQ.heatt,EQ.hstat,EQ.enet,EQ.enkat,EQ.jmazt,EQ.gzt,EQ.jezt), ftrj, HIGHEST_PROTOCOL)
    # backup of final conf
    write_input(EQ, N, nstep, conf_out=path+"conf_eq.b")
    # for visualization
    writexyz(EQ, N, filexyz=path+'conf_eq.xyz')
    return EQ



In [37]:
from numpy import zeros, pi, sin, cos, empty, array, float64
#from m_lj import LJ
#from m_write import *
from time import process_time
from  pickle import dump, load, HIGHEST_PROTOCOL

if __name__ == "__main__":
    # r-parameters # defaults
    dir="./RUN/"
    nstep= 10
    rho  = 0.984   #  Xe 0.634
    mx   = 8       #  8 Xe 5
    my   = 12      #  10 Xe 5
    mz   = 24      #  25 Xe 16
    zbuf = 4       #
    kt   = 0.79    # Xe 1.115
    dk0  = 0.00    # equilibrio
    dkt  = 0.10    # 0.20 Xe 0.11
    dt   = 0.005   # 0.005
    freq = 20      #  2000
    lther= 3       #  3 ( shifted by 2)
    gacc = 0.      # gravity acceleration (force with m=1)
    fwall= 1300.   # fixing lateral pressure
    rth2 = 2*(mz+1)#  lb=2*mz - shift
    my  += 2       # 
    mz  += zbuf    # 
    print( "# number of boxes mx = %d, my = %d, mz = %d" % (mx, my, mz) )
    print( "# mean (kinetic) temperature kt = %8.4f " % (kt) )
    print( "# integration time step dt = %8.4f" % dt )
    print( "# number of time steps for this run nstep = %d" % (nstep) )    
    t0 = process_time()
    IN = LJ(rho, gacc, mx, my, mz, zbuf, nstep*freq)
    IN.mode = 0
    IN.fwall = fwall
    print( "# creating EQ object, cpu = %8.4f" % (process_time()-t0) )
    print( "# density rho = %8.4f  (=%8.4f)" % (IN.rho, rho))
    print( "# temperature T = %8.4f  (dT=%8.4f)" % (kt, dkt))
    print( "# external conditions: gravity g=%8.4f" % gacc )
    print( "# external conditions: right wall force  Fw=%9.4f " % fwall )
    N = IN.npart
    rhofact = IN.lb / (IN.Lx * IN.Ly * IN.Lz)
    print( "# sides of the rectangular MD-box L = [ %8.4f %8.4f %8.4f ]" % (IN.Lx, IN.Ly, IN.Lz) )
    print( "# number of particles  N = %d" % N)
    print( "# potential cut-off radius  rcut = %8.4f *sigma" % IN.rcut)
    print( "# lateral size of slicing Deltaz = %8.4f" % (IN.Lz / (IN.lb)))

# number of boxes mx = 8, my = 14, mz = 28
# mean (kinetic) temperature kt =   0.7900 
# integration time step dt =   0.0050
# number of time steps for this run nstep = 10
# creating EQ object, cpu =   0.0097
# density rho =   0.9840  (=  0.9840)
# temperature T =   0.7900  (dT=  0.1000)
# external conditions: gravity g=  0.0000
# external conditions: right wall force  Fw=1300.0000 
# sides of the rectangular MD-box L = [  25.5353  44.6868  89.3737 ]
# number of particles  N = 73728
# potential cut-off radius  rcut =   3.0000 *sigma
# lateral size of slicing Deltaz =   1.5960


In [38]:
    # starting from (fcc) lattice
    IN.fcc()
    writexyz(IN, N, filexyz=dir+'fcc.xyz')

# lattice lenghts (ax,ay,az) = (1.5959586353904496, 1.5959586353904496, 1.5959586353904496)
# (mx, my, mz) = (16, 24, 48)
# number of lattice cells = 18432
# number of particles = 73728
# md-box sides [Lx, Ly, Lz ]= (25.535338166247193, 44.68684179093259, 89.37368358186518)
# end of initial fcc lattice construction npart = 73728


In [ ]:
    pas = 0
    mode=2
    t0 = process_time()    
    (enep, enek, ewg, vcmx, vcmy, vcmz) = IN.eqmdi( N, mx, my, mz, kt, pas, mode)
    print( "# initial conditions step = %d, cpu = %8.4f" % (pas, process_time()-t0) )
    print( "     'pas'    'enep'    'enek'       'enet' 'virial'    'fwl'    'fwr'    'zwr'       'vcm' ")
    etot = enep+enek+ewg
    print(" %9.2f %9.4f %9.4f %12.3f %8.4f %8.3f %8.2f %8.2f %7.1g %7.2g %7.2g" % (pas*dt,enep/N, enek/N, etot, ewg, IN.fwl, IN.fwr, IN.zwr, vcmx, vcmy, vcmz) )
    write_input(IN, N, nstep, conf_out=dir+"conf_fcc.b")

In [ ]:
    # Equilibrium run with temperature from maxwellian sampling
    print("# starting eqmd trajectory\n")
    print( "     'pas'    'enep'    'enek'       'enet'    'ewg'    'fwl'    'fwr'    'zwr'       'vcm' ")
    #etot = enep+enek
    nstep= 5    
    t0 = process_time()
    for it in range(nstep):
        (t, enep, enek, ewg, vcmx, vcmy, vcmz) = IN.eqmdr( N, mx, my, mz, kt, pas, dt, freq)
        etot = enep+enek+ewg
        pas += freq
        print(" %9.2f %9.4f %9.4f %12.3f %8.3f %8.2f %8.2f %8.3f %7.1g %7.2f %7.2f" % (pas*dt,enep/N, enek/N, etot, ewg, IN.fwl, IN.fwr, IN.zwr, vcmx, vcmy, vcmz) )
    print( "# (IN)-Equilibrium run, cpu = %8.4f" % (process_time()-t0) )
    g=3*N-1
    nstep *= freq
    print( "# ending initial equilibrium run  <ep>=%6.1f  <ek>=%7.4f   T=%8.3f  P=%8.3f\n" % ( IN.ept/nstep, IN.ekt/nstep, 2.*IN.ekt/(nstep*g), IN.pres/nstep) )
    print( "# ending initial equilibrium run  <Lload>=%8.2f  <Rload>=%8.2f   " % ( IN.lload/nstep, IN.rload/nstep) )
    print( "# ending initial equilibrium run  <Bload>=%8.2f  <Tload>=%8.2f \n" % ( IN.bload/nstep, IN.tload/nstep) )
    # final positions and momenta to file conf_in.b
    write_input(IN, N, nstep, conf_out=dir+"conf_eq.b")
    writexyz(IN, N, filexyz=dir+'conf1.xyz')

# starting eqmd trajectory

     'pas'    'enep'    'enek'       'enet'    'ewg'    'fwl'    'fwr'    'zwr'       'vcm' 


NameError: name 'process_time' is not defined

In [ ]:
    nrepeat = 4
    kt      = 0.79
    fwall   = 1300.
    gacc    = -0.01
    IN.mode = 0
    for repeat in range(nrepeat):
        nstep=11
        freq=200
        mode=2
        t0 = process_time()    
        pas = read_input(IN, N, conf_in=dir+"conf_eq.b")
        IN.fwall=fwall
        (enep, enek, ewg, vcmx, vcmy, vcmz) = IN.eqmdi( N, mx, my, mz, kt, pas, mode)
        print( "# initial conditions step = %d, cpu = %8.4f" % (pas, process_time()-t0) )    
        # Equilibrium run with temperature from maxwellian sampling
        print("# starting eqmd trajectory\n")
        print( "     'pas'    'enep'    'enek'       'enet'    'ewg'    'fwl'    'fwr'    'zwr'       'vcm' ")
        etot = enep+enek+ewg
        print(" %9.2f %9.4f %9.4f %12.3f %8.3f %8.2f %8.2f %8.3f %7.1g %7.2f %7.2f" % (pas*dt,enep/N, enek/N, etot, ewg, IN.fwl, IN.fwr, IN.zwr, vcmx, vcmy, vcmz) )
        t0 = process_time()
        for it in range(nstep):
            (t, enep, enek, ewg, vcmx, vcmy, vcmz) = IN.eqmdr( N, mx, my, mz, kt, pas, dt, freq)
            pas += freq    
            etot = enep+enek+ewg
            print(" %9.2f %9.4f %9.4f %12.3f %8.3f %8.2f %8.2f %8.3f %7.1g %7.2f %7.2f" % (pas*dt,enep/N, enek/N, etot, ewg, IN.fwl, IN.fwr, IN.zwr, vcmx, vcmy, vcmz) )
        print( "# (IN)-Equilibrium run, cpu = %8.4f \n" % (process_time()-t0) )
        nstep *= freq
        g = 3*N-2  # z-component of total momentum not conserved
        print( "# ending initial equilibrium run  <ep>=%6.1f  <ek>=%7.4f   T=%8.3f  P=%8.3f " % ( IN.ept/nstep, IN.ekt/nstep, 2.*IN.ekt/(nstep*g), IN.pres/nstep) )
        print( "# ending initial equilibrium run  <Lload>=%8.2f  <Rload>=%8.2f " % ( IN.lload/nstep, IN.rload/nstep) )
        print( "# ending initial equilibrium run  <Bload>=%8.2f  <Tload>=%8.2f \n" % ( IN.bload/nstep, IN.tload/nstep) )
        # final positions and momenta to file conf_in.b
        write_input(IN, N, pas, conf_out=dir+"conf_eq.b")
        writexyz(IN, N, filexyz=dir+'conf10.xyz')

 ??? reading 2304 particle configuration expected 6144


IndexError: index 2304 is out of bounds for axis 0 with size 2304

In [ ]:
%%file task-eq.py

from eqrun import *
if __name__ == "__main__":
    #dt=0.005
    #option=0
    nstep=500
    rho=0.984
    mx=6
    my=16
    mz=32
    zbuf=4
    kt=0.8
    dkt=0.0
    freq=200
    lther=3
    mode=2
    fwall = 2277.
    gacc=-0.01
    path="./RUN/"
    EQ = eqrun(nstep=nstep,rho=rho,mx=mx,my=my,mz=mz,zbuf=zbuf,kt=kt,dkt=dkt,fwall=fwall,gforce=gacc,freq=freq,lther=lther,mode=mode,path=path)

In [ ]:
%%file task-we.py

from nerun import *
if __name__ == "__main__":
    #dt=0.005
    nstep=1000
    rho=0.984
    mx=6
    my=16
    mz=32
    zbuf=4
    kt=0.79
    dkt=0.0
    freq=200
    lther=3
    mode=3
    option=0
    fwall = 2277.
    gacc= -0.01
    path="./RUN/"
    labelf="W"
    NE = nerun(nstep=nstep,rho=rho,mx=mx,my=my,mz=mz,zbuf=zbuf,kt=kt,dkt=dkt,fwall=fwall,gforce=gacc,freq=freq,lther=lther,mode=mode,option=option,path=path,labelf=labelf)

In [ ]:
%%file task-ne.py

from nerun import *
if __name__ == "__main__":
    nstep=2000
    rho=0.984
    mx=6
    my=16
    mz=32
    zbuf=4
    kt=0.79
    dkt=0.1
    freq=200
    lther=3
    mode=3
    option=0
    fwall = 2277.
    gacc = -0.01
    path="./RUN/"
    labelf="A"
    NE = nerun(nstep=nstep,rho=rho,mx=mx,my=my,mz=mz,zbuf=zbuf,kt=kt,dkt=dkt,fwall=fwall,gforce=gacc,freq=freq,lther=lther,mode=mode,option=option,path=path,labelf=labelf)

In [ ]:
%%file task-st.py

from nerun import *
if __name__ == "__main__":
    nstep=2000
    rho=0.984
    mx=6
    my=16
    mz=32
    zbuf=4
    kt=0.79
    dkt=0.1
    freq=200
    lther=3
    mode=3
    option=0
    fwall = 2277.
    gacc = -0.01
    path="./RUN/"
    labelf="B"
    NE = nerun(nstep=nstep,rho=rho,mx=mx,my=my,mz=mz,zbuf=zbuf,kt=kt,dkt=dkt,fwall=fwall,gforce=gacc,freq=freq,lther=lther,mode=mode,option=option,path=path,labelf=labelf)

### Equilibrium dynamics: crystal at constant Temperature and Volume or Pressure(moving lateral wall)

In [ ]:
%%bash 
  nohup time /opt/miniconda3/bin/python3 task-eq.py > RUN/task-eq.out &

In [ ]:
%%bash 
  nohup time /opt/miniconda3/bin/python3 task-we.py > RUN/task-neW.out &

### Non-equilibrium dynamics: Melting by constant Flux/Temperature heating at constant Pressure 

In [ ]:
%%bash 
  nohup time /opt/miniconda3/bin/python3 task-ne.py > RUN/task-neA.out &

In [ ]:
%%bash 
  nohup time /opt/miniconda3/bin/python3 task-st.py > RUN/task-neB.out &

#### all-in-one batch

In [ ]:
%%bash
  nohup bash batch.sh